### Imports

In [ ]:
import os
import time

import numpy as np
import pandas as pd
from pathlib import Path

import string
import itertools

import sys
sys.path.append("..")

from simulation.simulation_utils import simulate
from simulation.simulation_tools import get_optimal_sim_XY

rng = np.random.default_rng()

COL_NAMES = list(string.ascii_uppercase) + ["".join(a) for a in list(itertools.permutations(list(string.ascii_uppercase), r=2))]

### Single Simulation

In [ ]:
# data
data_path = list(Path(".").resolve().parents)[1] / 'data' / 'cp_style' / 'increasing_edges_cp_1' / 'data'
fn = os.listdir(data_path)[0]
true_data = pd.read_csv(data_path / fn)

# TCS configuration
cfg = {
    "cd_method" : "PCMCI", 
    "cd_kwargs" : None, 
    "pred_method" : "TCDF", 
    "pred_kwargs" : None, 
    "o_approximation" : "spline", 
    "noise_approximation" : "spline",
}

# simulate
start_time = time.time()
sim_data, sim_scm, funcs_and_noise, scores = simulate(
    true_data=true_data, 
    true_label=None, 
    n_samples=500, 
    verbose=True, 
    **cfg
)
elapsed_time = time.time() - start_time
print(f"LOG : Single Simulation : Elapsed time for single simulation: {round(elapsed_time, 2)}")

### Optimized Simulation

In [ ]:
# data
data_path = list(Path(".").resolve().parents)[1] / 'data' / 'fMRI' / 'timeseries'
fn = os.listdir(data_path)[0]
true_data = pd.read_csv(data_path / fn)

# simulate
start_time = time.time()
res = get_optimal_sim_XY(
        true_data = true_data, 
        CONFIGS = None, 
        done_eval = False,
        optimal_det_config = None,
        optimal_det_func = None, 
        sparsity_penalty=True,
        verbose = True
)
elapsed_time = time.time() - start_time
print(f"LOG : Optimized Simulation : Elapsed time for optimized simulation: {round(elapsed_time, 2)}")